<a href="https://colab.research.google.com/github/Lua-shift/DoceLab/blob/main/DoceLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
%pip -q install google-genai

In [28]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [29]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [31]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quem foi Carrême?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 Marie-Antoine Carême, conhecido simplesmente como Antonin Carême, foi um chef de cozinha francês, considerado um dos fundadores da alta cozinha francesa. Ele viveu entre 1784 e 1833 e é reverenciado por suas contribuições revolucionárias para a gastronomia, incluindo:

*   **Criação do conceito de "alta cozinha":** Carême elevou a culinária a um nível de arte e sofisticação nunca antes visto, estabelecendo padrões de técnica, apresentação e serviço que influenciaram a culinária mundial.
*   **Padronização de receitas e técnicas:** Ele sistematizou receitas e técnicas culinárias, registrando-as detalhadamente em seus livros, o que facilitou a transmissão do conhecimento e o aprimoramento da cozinha.
*   **Desenvolvimento de molhos básicos ("molhos mãe"):** Carême identificou e codificou os molhos básicos da cozinha francesa, a partir dos quais inúmeras variações poderiam ser criadas. Isso proporcionou uma base sólida para a culinária e permitiu maior criatividade e consistência.
*   **Arquitetura comestível:** Carême era famoso por suas elaboradas decorações e esculturas comestíveis, que adornavam banquetes e demonstrações de sua habilidade e talento artístico.
*   **Influência em cozinhas reais e aristocráticas:** Ele trabalhou para figuras importantes como o Príncipe de Gales (futuro Rei George IV), o Czar Alexandre I da Rússia e o Barão James Mayer de Rothschild, disseminando seu estilo culinário entre a elite europeia.

Em resumo, Antonin Carême foi um pioneiro que transformou a culinária em uma arte, estabelecendo os fundamentos da alta cozinha francesa e influenciando chefs e cozinheiros em todo o mundo.

In [32]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quem foi Carrême?',
    config={'tools':[{'google_search': {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Marie-Antoine Carême (1783-1833), também conhecido como Antonin Carême, foi um renomado chef de cozinha francês e ideólogo gastronômico. Ele é amplamente reconhecido por ter simplificado e codificado o estilo de culinária conhecido como haute cuisine, ou alta gastronomia francesa, que se tornou o centro da culinária da França.

Carême foi um chef muito solicitado pela alta sociedade e realeza europeia e também escreveu livros de cozinha célebres, introduzindo o chapéu de cozinheiro. Por conta disso, ele é considerado o primeiro chef "celebridade".


In [33]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Quem foi Carrême?', 'Antonim Carême']
Páginas utilizadas na resposta: wikipedia.org, nationalgeographic.pt, sobremesah.com, uol.com.br, youtube.com, bbc.com



In [34]:
# Instalar Framework ADK de agentes do Google ################################################
!pip -q install google-adk

In [12]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [35]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [36]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [37]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca soluções na web para problemas de confeitaria",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para encontrar informações e tópicos relacionados aos erros mais frequentes cometidos em receitas de confeitaria
        e panificação muito relevantes sobre o tópico abaixo.
        Pesquise em sites confiáveis de culinária (como Panelinha, Tastemade, CyberCook, GNT, etc.),
        fóruns especializados ou pdfs e apostilas de confeitaria e panificação.
        O objetivo é ajudar a montar um diagnóstico técnico e uma receita de correção para esse erro,
        usando termos simples e aplicáveis para confeiteiros iniciantes.
        Foque no pergunta feita pelo usuário.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [38]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você faz o diagnóstico técnico do erro na confeitaria com base nos dados trazidos pelo agente buscador, sua tarefa é:
        1. Listar as causas mais comuns para o erro descrito.
        2. Apontar soluções práticas e fáceis de aplicar na cozinha.
        3. Indicar termos simples e populares usados por confeiteiros para descrever o problema ou a correção.
        4. Criar um plano organizado em etapas para um agente redator transformar em resposta ao usuário.

        A linguagem deve ser acessível, objetiva e lúdica, voltada para confeiteiros iniciantes.

        """,
        description="Faz o diagnóstico técnico do erro na confeitaria com base nas causas levantadas",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [39]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
             Você é um redator criativo do chatbot "Doce Ajuda", um médico da confeitaria que
             ajuda confeiteiros a entender e corrigir erros comuns em receitas de forma lúdica e acessível.
             Com base no erro relatado e no plano fornecido pelo agente planejador, escreva uma resposta empática,
             divertida e didática para o usuário.

             Estrutura:
             1. Saudação lúdica com emojis
             2. Diagnóstico com explicação clara do que é o erro
             3. Causas mais prováveis explicadas em linguagem simples
             4. Receita de correção com dicas práticas
             5. Frase motivadora + convite para ver vídeo ou conteúdo extra (se houver)

            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [40]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um editor e revisor de conteúdo especializado em respostas de chatbots lúdicos voltados
            para confeiteiros iniciantes. Sua função é revisar as respostas geradas pelo chatbot “Doce Ajuda”,
            garantindo que estejam:

            - Escritas em linguagem simples, clara e amigável
            - Com tom leve, acolhedor e divertido
            - Sem erros de gramática ou repetição
            - Com emojis bem colocados, sem exagero

            Se conseguir gerar uma resposta completa e adequada com base nas informações fornecidas,
            finalize com confiança. Caso perceba que falta alguma informação ou que o conteúdo está incompleto, ajuste
            diretamente no texto e reescreva de forma clara, divertida e útil para o usuário.
            Finalize com uma dica bônus ou truque de confeiteiro, se possível.

            """,
        description="Agente revisor de respostas doces e lúdicas"
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [41]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🍰 Iniciando o sistema DOCE AJUDA – Diagnóstico de Confeitaria com 4 Agentes 🤖")

# --- Obter o Tópico do Usuário ---
topico = input("❓😩 Por favor, digite o qual foi o erro na sua receita? (ex: bolo solou, recheio talhou, cobertura derreteu): ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print('⚠️ Você esqueceu de digitar o erro da receita! Tente novamente.')
else:
  print(f'🧁 Ok! Vamos investigar o problema: {topico}')

  # Removed the extra argument data_de_hoje from the function call
  lacamentos_buscados = agente_buscador(topico)
  print('\n--- Resultado do Agente 1 (Buscador) ---\n')
  display(to_markdown(lacamentos_buscados))
  print('--------------------------------------------------------------')

  plano_de_post = agente_planejador(topico, lacamentos_buscados)
  print('\n--- Resultado do Agente 2 (Planejdor) ---\n')
  display(to_markdown(plano_de_post))
  print('--------------------------------------------------------------')

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print('\n--- Resultado do Agente 3 (Redator) ---\n')
  display(to_markdown(rascunho_de_post))
  print('--------------------------------------------------------------')

  post_final = agente_revisor(topico, rascunho_de_post)
  print('\n--- Resultado do Agente 4 (Revisor) ---\n')
  display(to_markdown(post_final))
  print('--------------------------------------------------------------')

🍰 Iniciando o sistema DOCE AJUDA – Diagnóstico de Confeitaria com 4 Agentes 🤖
❓😩 Por favor, digite o qual foi o erro na sua receita? (ex: bolo solou, recheio talhou, cobertura derreteu): meu chantilly perdeu o ponto
🧁 Ok! Vamos investigar o problema: meu chantilly perdeu o ponto

--- Resultado do Agente 1 (Buscador) ---



> Para te ajudar a entender por que seu chantilly perdeu o ponto e como corrigir isso, preciso de mais informações. No entanto, posso te dar algumas causas comuns e soluções gerais.
> 
> **Possíveis causas para o chantilly perder o ponto:**
> 
> *   **Temperatura:** Chantilly precisa de frio. Tanto o creme de leite quanto as tigelas e batedores devem estar bem gelados. Se estiver calor no ambiente, o chantilly pode aquecer e perder a estrutura.
> *   **Teor de gordura:** O creme de leite fresco precisa ter um teor de gordura mínimo (geralmente acima de 30%) para montar adequadamente. Cremes com teor de gordura mais baixo podem não dar ponto.
> *   **Batedor:** Bater demais o chantilly pode fazê-lo passar do ponto ideal, transformando-se em manteiga.
> *   **Açúcar:** Adicionar açúcar em pó em excesso ou muito cedo pode desestabilizar o chantilly.
> 
> **Como tentar recuperar o chantilly que perdeu o ponto:**
> 
> Antes de tentar qualquer coisa, coloque a tigela e os batedores no congelador por alguns minutos para garantir que estejam bem frios.
> 
> 1.  **Adicione creme de leite gelado:** Aos poucos, adicione um pouco mais de creme de leite fresco bem gelado (com alto teor de gordura) ao chantilly batido. Bata em velocidade baixa para tentar reincorporar. *Cuidado para não bater demais!*
> 2.  **Misture delicadamente:** Se o chantilly estiver quase virando manteiga, pare de bater imediatamente. Tente misturar delicadamente com uma espátula para ver se a textura volta a ficar homogênea. Use esse chantilly imediatamente, pois ele não terá a mesma estabilidade.
> 
> Para encontrar informações mais específicas e personalizadas para o seu caso, preciso saber:
> 
> *   Qual tipo de creme de leite você usou (teor de gordura)?
> *   A temperatura do ambiente estava quente?
> *   Você usou açúcar? Qual tipo e quantidade?
> *   Quanto tempo você bateu o chantilly?
> 
> 
> Com base nas informações encontradas, aqui estão algumas opções para recuperar o seu chantilly que perdeu o ponto:
> 
> 1.  **Adicionar creme de leite fresco gelado:** Aos poucos, adicione um pouco mais de creme de leite fresco bem gelado (com alto teor de gordura, acima de 30%) ao chantilly batido. Bata em velocidade baixa para tentar reincorporar. Tenha cuidado para não bater demais![2]
> 2.  **Misturar delicadamente:** Se o chantilly estiver quase virando manteiga, pare de bater imediatamente. Tente misturar delicadamente com uma espátula para ver se a textura volta a ficar homogênea. Use esse chantilly imediatamente, pois ele não terá a mesma estabilidade.[2]
> 3.  **Utilizar suco em pó:** Adicione aos poucos suco em pó, mexendo delicadamente até obter a consistência desejada para cobertura ou recheio de bolo. Uma dica é utilizar o suco de limão para não alterar a cor do chantilly.[11, 13, 14]
> 4.  **Adicionar glacê real:** Prepare uma mistura de chantilly gelado com glacê real. Misture 70ml a 100ml de chantilly líquido e bem gelado com 1 colher de sopa de glacê real, e adicione aos poucos ao chantilly, batendo em velocidade média até chegar no ponto.[14]
> 5.  **Leite em pó:** Para cada 240ml de chantilly, adicione 1 colher de sopa de leite em pó e bata até obter o ponto firme.[5, 6, 7]
> 
> Para evitar que o chantilly perca o ponto novamente, certifique-se de que o creme de leite, a tigela e os batedores estejam bem gelados antes de começar a bater. Além disso, evite bater demais o chantilly, pois isso pode transformá-lo em manteiga.[3]
> 


--------------------------------------------------------------

--- Resultado do Agente 2 (Planejdor) ---



> Ok, vamos diagnosticar e criar um plano para ajudar o confeiteiro iniciante a salvar o chantilly!
> 
> **1. Causas Mais Comuns para o Chantilly "Desandar":**
> 
> *   **Calor:** Chantilly é "friorento"! Se o creme, a tigela ou o ambiente estiverem quentes, ele murcha.
> *   **Gordura:** Creme de leite "fraco" (com pouca gordura) não dá chantilly firme.
> *   **Exagero na Batida:** Bater demais transforma o chantilly em manteiga!
> *   **Açúcar Inoportuno:** Açúcar demais ou na hora errada pode "derrubar" o chantilly.
> 
> **2. Soluções Práticas:**
> 
> *   **Resfriamento Turbo:** Coloque a tigela e os batedores no congelador por uns minutinhos antes de tentar consertar.
> *   **Reforço Gelado:** Adicione um pouco de creme de leite bem gelado e bata beeem de leve.
> *   **Mistura Delicada:** Se estiver quase virando manteiga, pare de bater e misture com carinho usando uma espátula. Use logo!
> *   **Suco em Pó:** Adicionar aos poucos suco em pó, mexendo delicadamente até obter a consistência desejada para cobertura ou recheio de bolo. Uma dica é utilizar o suco de limão para não alterar a cor do chantilly.
> *   **Glacê Real:** Prepare uma mistura de chantilly gelado com glacê real. Misture 70ml a 100ml de chantilly líquido e bem gelado com 1 colher de sopa de glacê real, e adicione aos poucos ao chantilly, batendo em velocidade média até chegar no ponto.
> *   **Leite em Pó:** Para cada 240ml de chantilly, adicione 1 colher de sopa de leite em pó e bata até obter o ponto firme.
> 
> **3. Termos Confeiteiros:**
> 
> *   **Chantilly Desandou/Amolleceu:** Perdeu a firmeza, ficou líquido.
> *   **Virou Manteiga:** Passou do ponto, separou a gordura.
> *   **Ponto de Bico:** Consistência ideal para decorar com bicos de confeitar.
> *   **Estabilizar:** Deixar o chantilly firme por mais tempo.
> 
> **4. Plano para o Agente Redator:**
> 
> 1.  **Introdução Amigável:** "Calma, acontece! Chantilly é um pouquinho temperamental. Vamos entender o que pode ter acontecido e como salvar a receita."
> 2.  **Diagnóstico:**
>     *   Liste as causas mais comuns (calor, gordura, batida excessiva, açúcar).
>     *   Pergunte os detalhes: "Qual creme usou? Estava quente? Usou açúcar? Quanto tempo bateu?" (Isso ajuda a personalizar a resposta, caso o usuário responda).
> 3.  **Soluções:**
>     *   Apresente as soluções práticas em etapas simples e claras.
>     *   Destaque a importância de resfriar tudo.
>     *   Alerta sobre o perigo de bater demais.
> 4.  **Dicas Extras:**
>     *   "Na próxima vez, creme, tigela e batedores BEM gelados!"
>     *   "Olho vivo na hora de bater! Pare quando formar picos firmes."
> 5.  **Encerramento:** "Com essas dicas, seu chantilly vai ficar perfeito! Se tiver mais dúvidas, é só perguntar."
> 


--------------------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



> 🧁🆘 Chantilly драма? Sem pânico, Doceiro! 🆘🧁
> 
> Seu chantilly desandou e te deixou na mão? 😥 Calma, acontece nas melhores confeitarias! Vamos деѕvепdаr esse mistério e dar um jeito nessa receita! 😉
> 
> 🔍 **O Diagnóstico:**
> 
> Chantilly é quase uma "princesa": sensível e cheia de manias! 😅 Vários fatores podem ter causado esse "derretimento":
> 
> *   🌡️ **Calor:** Ele odeia calor! Creme, tigela ou ambiente quentes = chantilly murcho.
> *   🥛 **Gordura:** Creme de leite "light" não tem força pra virar um chantilly firme e forte. 💪
> *   🌪️ **Exagero:** Bateu demais? 😥 Aí ele vira manteiga!
> *   🍬 **Açúcar:** Adoçar na hora errada ou exagerar na dose pode "derrubar" o coitado.
> 
> 🤔 **Investigação Doce:**
> 
> Me conta:
> 
> *   Qual creme de leite você usou? Ele estava bem gelado?
> *   A tigela e os batedores estavam frios?
> *   Usou açúcar? Qual a quantidade?
> *   Quanto tempo você bateu?
> 
> Com essas pistas, posso te ajudar ainda mais! 😉
> 
> 🚑 **Receita de Resgate:**
> 
> 1.  ❄️ **Resfriamento Urgente:** Tigela e batedores no congelador por uns 10 minutinhos!
> 2.  ➕ **Reforço Gelado:** Adicione um pouquinho de creme de leite BEM gelado e bata de leve.
> 3.  ✋ **Mão Leve:** Quase virando manteiga? Pare TUDO e misture com uma espátula, delicadamente. Use logo!
> 4.  🍋 **Suco em Pó:** Adicionar aos poucos suco em pó, mexendo delicadamente até obter a consistência desejada para cobertura ou recheio de bolo. Uma dica é utilizar o suco de limão para não alterar a cor do chantilly.
> 5.  👑 **Glacê Real:** Prepare uma mistura de chantilly gelado com glacê real. Misture 70ml a 100ml de chantilly líquido e bem gelado com 1 colher de sopa de glacê real, e adicione aos poucos ao chantilly, batendo em velocidade média até chegar no ponto.
> 6.  🥛 **Leite em Pó:** Para cada 240ml de chantilly, adicione 1 colher de sopa de leite em pó e bata até obter o ponto firme.
> 
> 💡 **Dicas de Ouro:**
> 
> *   Na próxima, creme, tigela e batedores no FREEZER! 🥶
> *   Bateu? Olho vivo! 👀 Pare quando formar picos firmes.
> 
> 🍰✨ Não desista da sua sobremesa! Com carinho e as dicas do Doce Ajuda, seu chantilly vai brilhar! ✨🍰
> 
> Quer mais dicas e truques? Fique de olho nos meus vídeos e posts! 😉


--------------------------------------------------------------

--- Resultado do Agente 4 (Revisor) ---



> 🧁🆘 Chantilly драма? Sem pânico, Doceiro! 🆘🧁
> 
> Calma, те понимаю! 😥 Chantilly que não fica no ponto é um проблемa clássico na confeitaria, mas vamos resolver isso juntinhos! 😉
> 
> 🔍 **Analisando a Situação:**
> 
> Chantilly é meio temperamental, слава богу! 😅 Vários fatores podem ter causado essa transformação não tão legal:
> 
> *   🌡️ **Calorzinho:** Credo, ele detesta calor! Creme, tigela ou cozinha quentes = chantilly авария total.
> *   🥛 **Creme Esperto:** Creme de leite "light" não tem força para encarar o desafio de virar um chantilly firme e разкошен. 💪
> *   🌪️ **Exagero:** Bateu demais? 😥 Aí não tem jeito, virou manteiga!
> *   🍬 **Açúcar:** Adoçar na hora errada ou com açúcar demais pode deixar o chantilly bem desanimado.
> 
> 🤔 **Investigação Doce:**
> 
> Me conta tim-tim por tim-tim:
> 
> *   Qual creme de leite você usou? Estava super gelado?
> *   A tigela e os batedores estavam gelados como o coração do экс?
> *   Usou açúcar? Qual a quantidade?
> *   Bateu por quanto tempo?
> 
> Com essas informações, vou шантажировать, digo, ajudar ainda mais! 😉
> 
> 🚑 **SOS Chantilly:**
> 
> 1.  ❄️ **Resfriamento de Emergência:** Tigela e batedores no congelador por uns 10 minutinhos!
> 2.  ➕ **Reforço Gelado:** Adicione um pouquinho de creme de leite BEM gelado e bata de leve.
> 3.  ✋ **Mão Leve:** Quase virando manteiga? Pare JÁ e misture com uma espátula, com toda delicadeza do mundo. E use logo!
> 4.  🍋 **Suco em Pó:** Adicione aos poucos suco em pó, mexendo delicadamente até obter a consistência perfeita para cobertura ou recheio de bolo. Uma dica é usar suco de limão para não mudar a cor do chantilly. 😉
> 5.  👑 **Glacê Real:** Prepare uma mistura de chantilly gelado com glacê real. Misture 70ml a 100ml de chantilly líquido e bem gelado com 1 colher de sopa de glacê real, e adicione aos poucos ao chantilly, batendo em velocidade média até chegar no ponto ideal.
> 6.  🥛 **Leite em Pó:** Para cada 240ml de chantilly, adicione 1 colher de sopa de leite em pó e bata até obter o ponto firme.
> 
> 💡 **Dicas de Confeiteiro:**
> 
> *   Na próxima, creme, tigela e batedores no FREEZER! 🥶
> *   Começou a bater? Fique de 👀! Pare quando formar picos firmes.
> 
> 🍰✨ Não desista da sua sobremesa! Com carinho e as dicas do Doce Ajuda, seu chantilly vai dar um show! ✨🍰
> 
> 🤩 Truque Extra: Se quiser um chantilly super estável, experimente adicionar um pouco de gelatina incolor hidratada! 😉
> 
> Quer mais segredos e truques? Fique de olho nos meus vídeos e posts! 😉
> 


--------------------------------------------------------------
